In [1]:
import pandas as pd
import os
import math
from datetime import datetime
import numpy as np
from numpy import argmax

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import sys

from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from time import time
from sklearn.metrics import classification_report,matthews_corrcoef
from copy import deepcopy
from sklearn.utils import shuffle

# path to the dataset
# file = 'Shaleeza.Dataset.v1.csv'
file = 'IoT Network Intrusion Dataset.csv'
data = pd.read_csv(file)

In [5]:
def data_preprocessing(targets_others):
    dataset = pd.read_csv(file, error_bad_lines=False, low_memory=False)
    dataset = dataset.drop(['Flow_ID', 'Src_IP', 'Dst_IP', 'Dst_Port', 'Protocol'], axis=1)
    dataset = dataset.drop(['Timestamp'], axis=1)

    # contain only single values
    dataset = dataset.drop(
        ['Fwd_PSH_Flags', 'Fwd_URG_Flags', 'Fwd_Byts/b_Avg', 'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg',
         'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Init_Fwd_Win_Byts', 'Fwd_Seg_Size_Min'], axis=1)

    dataset['Flow_Byts/s'] = round(dataset['Flow_Byts/s'],2)

    dataset = dataset.drop(targets_others, axis=1)

    dataset = dataset.reset_index()
    dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
    dataset.dropna(inplace=True)

    # correlation
    correlated_features = set()
    correlation_matrix = dataset.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) >= 0.7:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)

    dataset.drop(labels=correlated_features, axis=1, inplace=True)

    return dataset

def kFoldCV(model, data, n_fold=10):
    diff = int(len(data)/n_fold)
    results = np.zeros((1, 4))
    predictY = deepcopy(data[:,-1])
    for i in range(n_fold):
        begin = diff*i
        end = diff*(i+1)
#         if i == n_fold-1:
#             end = -1
        test = data[begin:end]
        train = deepcopy(data)
        train = np.delete(train, range(begin, end),axis=0)
        X_train, y_train = InstanceHardnessThreshold().fit_resample(train[:,:-1], train[:,-1])
        y_train = y_train.astype('int').reshape(len(X_train))
        predictY[begin:end] = model.fit(X_train, y_train).predict(test[:,:-1])
    t = classification_report(data[:,-1], predictY)
    print(t)
    print(matthews_corrcoef(data[:,-1], predictY))
        
#         results = results + getResults(model, train[:,:-1], train[:,-1],test[:,:-1],test[:,-1])
#     return results/n_fold

def kFoldCV(model, data, n_fold=10):
    diff = int(len(data)/n_fold)
    results = np.zeros((1, 4))
    predictY = data[:,-1].astype('int')
    targetY = deepcopy(predictY)
#     predictY = deepcopy(data[:,-1]).astype('int')
    
    for i in range(n_fold):
        begin = diff*i
        end = diff*(i+1)
#         if i == n_fold-1:
#             end = -1
        test = data[begin:end]
        train = deepcopy(data)
        train = np.delete(train, range(begin, end),axis=0)
        X_train, y_train = train[:,:-1], train[:,-1].astype('int')
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(test[:,:-1])
#         X_train, y_train =  SMOTE().fit_resample(train[:,:-1], y_train.astype('int'))#.astype('int')
        

        predictY[begin:end] = model.fit(X_train, y_train).predict(X_test)
#         targetY[begin:end] = test[:,-1].astype('int')
#         print(predictY)
#         print(targetY)
#         predictY[begin:end] = model.fit(X_train, y_train).predict(X_test)
    t = classification_report(targetY.astype('int'), predictY.astype('int'))
    print(t)
    print(matthews_corrcoef(data[:,-1].astype("int"), predictY.astype("int")))


def getResults(model, X_train, y_train,X_test,y):
    predictY = model.fit(X_train, y_train).predict(X_test)
    t = classification_report(y, predictY)#, target_names=['0', '1', '2']
    return t


In [3]:
data = data_preprocessing(['Label', 'Cat'])
data = data.values
data = data[:,1:]
X_train, y_train = data[:,:-1], data[:,-1]
y_train = LabelEncoder().fit_transform(y_train)

data = np.c_[X_train,y_train]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21752\2019069928.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataset = pd.read_csv(file, error_bad_lines=False, low_memory=False)


In [6]:
t = time()
length = [5]
for i in length:
    model = DecisionTreeClassifier(max_depth=i)
    kFoldCV(model, data)


print(time()-t)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     59391
           1       0.00      0.00      0.00     35377
           2       0.00      0.00      0.00     55124
           3       1.00      0.00      0.00     55818
           4       0.70      0.59      0.64    121178
           5       0.58      0.97      0.72    183189
           6       0.84      0.80      0.81     40073
           7       0.00      0.00      0.00     22192
           8       0.43      0.94      0.59     53073

    accuracy                           0.62    625415
   macro avg       0.50      0.48      0.42    625415
weighted avg       0.58      0.62      0.53    625415

0.554890173756306
136.41163444519043


In [7]:
t = time()
hidden_layer_sizes  = [200]
max_iter = [300]
for i in hidden_layer_sizes:
    for j in max_iter:
        model = MLPClassifier(hidden_layer_sizes=i, max_iter=j)
        kFoldCV(model, data)


print(time()-t)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\anaconda3\lib\sit

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59391
           1       0.75      0.78      0.77     35377
           2       0.32      0.24      0.27     55124
           3       0.34      0.23      0.27     55818
           4       0.72      0.83      0.77    121178
           5       0.76      0.82      0.79    183189
           6       0.92      0.89      0.90     40073
           7       0.62      0.05      0.10     22192
           8       0.62      0.84      0.71     53073

    accuracy                           0.71    625415
   macro avg       0.67      0.63      0.62    625415
weighted avg       0.69      0.71      0.69    625415

0.6535031515886706
22767.330097913742


In [8]:
t = time()
n_estimators  = [200]
max_depth = [5]
for i in n_estimators:
    for j in max_depth:
        model = RandomForestClassifier(n_estimators=i, max_depth=j)
        kFoldCV(model, data)

print(time()-t)


C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59391
           1       0.91      0.16      0.27     35377
           2       0.21      0.00      0.00     55124
           3       0.64      0.02      0.05     55818
           4       0.44      0.92      0.59    121178
           5       0.65      0.92      0.76    183189
           6       0.85      0.83      0.84     40073
           7       0.00      0.00      0.00     22192
           8       0.79      0.05      0.10     53073

    accuracy                           0.61    625415
   macro avg       0.61      0.43      0.40    625415
weighted avg       0.62      0.61      0.51    625415

0.539159303436637
1479.8255393505096


In [13]:
# def kFoldCV(model, data, n_fold=10):
#     diff = int(len(data)/n_fold)
#     results = np.zeros((1, 4))
#     predictY = deepcopy(data[:,-1])
#     for i in range(n_fold):
#         begin = diff*i
#         end = diff*(i+1)
# #         if i == n_fold-1:
# #             end = -1
#         test = data[begin:end]
#         train = deepcopy(data)
#         train = np.delete(train, range(begin, end),axis=0)
#         X_train, y_train = InstanceHardnessThreshold().fit_resample(train[:,:-1], train[:,-1])
#         y_train = y_train.astype('int').reshape(len(X_train), 1)
#         predictY[begin:end] = model.fit(X_train, y_train).predict(test[:,:-1])
#     t = classification_report(data[:,-1], predictY)
#     print(t)
#     print(matthews_corrcoef(data[:,-1], predictY))
    

    
def kFoldCV(model, data, n_fold=10):
    diff = int(len(data)/n_fold)
    results = np.zeros((1, 4))
    predictY = deepcopy(data[:,-1])
    for i in range(n_fold):
        begin = diff*i
        end = diff*(i+1)
#         if i == n_fold-1:
#             end = -1
        test = data[begin:end]
        train = deepcopy(data)
        train = np.delete(train, range(begin, end),axis=0)
        X_train, y_train = InstanceHardnessThreshold().fit_resample(train[:,:-1], train[:,-1].astype(int))
#         print(train[:,-1].shape)
#         y_train = y_train.astype('int').reshape(len(X_train),1)
        model.fit(X_train, y_train)
        predictY[begin:end] = model.predict(test[:,:-1])
#         predictY[begin:end] = model.fit(train[:,:-1], train[:,-1].astype("int")).predict(test[:,:-1])
    t = classification_report(data[:,-1].astype("int"), predictY.astype("int"))
    print(t)
    print(matthews_corrcoef(data[:,-1].astype("int"), predictY.astype("int")))

In [14]:
t = time()
n_estimators  = [100]
max_depth = [4]
for i in n_estimators:
    for j in max_depth:
        model = XGBClassifier(n_estimators=i, max_depth=j,objective='mlogloss')
        kFoldCV(model, data)


print(time()-t)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59391
           1       0.94      0.87      0.90     35377
           2       0.28      0.48      0.35     55124
           3       0.28      0.39      0.32     55818
           4       0.96      0.81      0.88    121178
           5       1.00      0.70      0.83    183189
           6       0.99      0.86      0.92     40073
           7       0.31      0.85      0.45     22192
           8       0.88      0.53      0.66     53073

    accuracy                           0.71    625415
   macro avg       0.74      0.72      0.70    625415
weighted avg       0.83      0.71      0.75    625415

0.6752439350563558
7658.575343608856


In [16]:
def kFoldCV(model, data, n_fold=10):
    diff = int(len(data)/n_fold)
    results = np.zeros((1, 4))
    predictY = deepcopy(data[:,-1]).reshape(len(data[:,-1]),1)
    for i in range(n_fold):
        begin = diff*i
        end = diff*(i+1)
#         if i == n_fold-1:
#             end = -1
        test = data[begin:end]
        train = deepcopy(data)
        train = np.delete(train, range(begin, end),axis=0)
        X_train, y_train = InstanceHardnessThreshold().fit_resample(train[:,:-1], train[:,-1].astype(int))
#         print(train[:,-1].shape)
#         y_train = y_train.astype('int').reshape(len(X_train),1)
        model.fit(X_train, y_train)
        predictY[begin:end] = model.predict(test[:,:-1]).reshape(len(test[:,-1]),1)
#         predictY[begin:end] = model.fit(train[:,:-1], train[:,-1].astype("int")).predict(test[:,:-1])
    t = classification_report(data[:,-1].astype("int"), predictY.astype("int"))
    print(t)
    print(matthews_corrcoef(data[:,-1].astype("int"), predictY.astype("int")))

In [ ]:
t = time()
n_estimators  = [200]
max_depth = [5]
for i in n_estimators:
    for j in max_depth:
        model = CatBoostClassifier(n_estimators=i, max_depth=j,verbose=False)
        kFoldCV(model, data)

print(time()-t)